In [1]:
import math 
import random 
import numpy

In [16]:
# Example of random sample method
random.sample(range(5), 2)  # Picks two random UNIQUE numbers from a set of numbers -- use this for swap

[0, 1]

In [18]:
# Draws random number between 0 and 1 -- use for probability of accepting worse route
random.random()

0.7763942872791783

In [19]:
def simulated_annealing_tsp(cities, distances,
                            T=1000, T_min=1e-3, alpha=0.9, max_iter=100000):
    """
    Solves the Traveling Salesman Problem (TSP) using Simulated Annealing.

    Parameters:
        cities (list of str): List of city names.
        distances (list or array-type): 2D list (matrix) of distances between cities.
        T (float, optional): Initial temperature for annealing.
        T_min (float, optional): Minimum temperature to stop the process.
        alpha (float, optional): Cooling rate (should be close to 1, e.g., 0.995).
        max_iter (float, optional): Maximum allowed number of iterations.

    Returns:
        best_route (list of str): List of city names representing the best route found.
        best_distance (float): Total distance of the best route.
        temperature_at_best (float): Temperature at which the best route was found.
    """

    n = len(cities)  # Number of cities
    city_indices = list(range(n))  # City indices [0, 1, 2, ..., n-1]

    # --- Helper Functions ---

    def route_distance(route_indices):
        """
        Calculates the total distance of a given route (list of city indices),
        including the return trip back to the starting city.
        
        Parameters:
            route_indices (list of int): List of city indices representing the route.

        Returns:
            float: Total distance of the route, including the return trip to the starting city.
        """
        return sum(distances[route_indices[i]][route_indices[i + 1]] for i in range(n - 1)) \
               + distances[route_indices[-1]][route_indices[0]] # Adds distance from last city to start city

    def propose_new_route(route):
        """
        Proposes a neighboring route by swapping two randomly selected cities.
        
        Parameters: 
            route (list of int): Current route represented as a list of city indices.

        Returns:
            (list of int): New route with two cities swapped.
        """
        new_route = route.copy()  # Copy in case we reject new route
        i, j = random.sample(range(n), 2)
        new_route[i], new_route[j] = new_route[j], new_route[i]
        return new_route

    # --- Initialization ---

    current_route = city_indices.copy()  
    random.shuffle(current_route)  # Start with a random route
    current_distance = route_distance(current_route)  # Cost / "energy" of initial solution

    # Best known solution
    best_route = current_route.copy()  # Assume for now that first route is best
    best_distance = current_distance   # Update if we find better values of distance
    temperature_at_best = T  # T0, update if we find better routes during cooling step
    iteration = 0  # might be useful to keep track of for plottinh later

    # --- Main Annealing Loop ---

    while T > T_min and iteration < max_iter:
        iteration += 1

        # Propose a new route by swapping two cities
        candidate_route = propose_new_route(current_route)
        candidate_distance = route_distance(candidate_route)

        # Calculate energy difference
        delta_E = candidate_distance - current_distance

        # Apply Metropolis acceptance rule, check if new state has higher or lower energy
        if delta_E < 0:
            # Better solution: accept automatically
            current_route = candidate_route
            current_distance = candidate_distance
        else:
            # Worse solution: accept with a certain probability (more exploration at higher T)
            acceptance_probability = math.exp(-delta_E / T)
            if random.random() < acceptance_probability:
                current_route = candidate_route
                current_distance = candidate_distance
            # Else reject move and stay with the current route

        # Update the best solution found so far
        if current_distance < best_distance:
            best_distance = current_distance
            best_route = current_route[:]
            temperature_at_best = T

        # Cool down the system
        T *= alpha

    # --- Wrap-up ---

    # Convert best route from indices back to city names
    best_route_named = [cities[idx] for idx in best_route] + [cities[best_route[0]]]

    return best_route_named, best_distance, temperature_at_best


In [20]:
# Example 

cities = ['Strasbourg', 'Nancy', 'Paris', 'Mulhouse', 'Dijon', 'Besancon']
distances = [
    [0, 116, 398, 97, 245, 197],
    [116, 0, 281, 136, 174, 163],
    [398, 281, 0, 389, 263, 327],
    [97, 136, 389, 0, 178, 114],
    [245, 174, 263, 178, 0, 75],
    [197, 163, 327, 114, 75, 0]
]

best_route, best_distance, temp_at_best = simulated_annealing_tsp(cities, distances)

print("Optimal Route:", " -> ".join(best_route))
print(f"Total Distance: {best_distance:.2f} km")
print(f"Found at Temperature: {temp_at_best:.4f}")


Optimal Route: Paris -> Dijon -> Besancon -> Mulhouse -> Strasbourg -> Nancy -> Paris
Total Distance: 946.00 km
Found at Temperature: 1000.0000
